In [1]:
###worksfor hypo only as calculated for 'confidence' 

#### need to handle, low mapq and dup

import pandas as pd
from statistics import mean
import time
import numpy as np
from collections import defaultdict
import sys

start_time = time.time()

smfile='/Users/irffanalahi/Research/Research_code/gitignorefolder/MHB_per_read/softreadcounting/testtheidea/try2/try2_efficient/BL14_all_matrixCin_nr0.4_imputed_rowmean.txt_bg_intesectedwith_CD4DMRofBL14atleast.2SM_g1_CD4_3_g2_others_33.txt_othermean.txt'
finalfile='/Users/irffanalahi/Research/Research_code/gitignorefolder/MHB_per_read/softreadcounting/testtheidea/try2/try2_efficient/CD4allrange_NR_81205_insilmix41_sorted_howsm_single_mode_pp_masterdf.pkl'

acccol='hypolist'
noACCcol='hyperlist'

smdf=pd.read_csv(smfile,sep="\t",index_col=['position'])
finaldf=pd.read_pickle(finalfile)

finaldf=finaldf[~finaldf['assignedcelltype'].isin(['lowmapq', 'DupRead'])]

deltagreaterforpositive=0  ##############>0

deltasmallerfornegative=0 ############<0

smdf.head()

,CD4,CD8,nB,NK,Mn,mNeu,m8,DC,Eo,Tr,...,Mn-others,mNeu-others,m8-others,DC-others,Eo-others,Tr-others,m4-others,Mg-others,Er-others,mB-others
position,,,,,,,,,,,,,,,,,,,,,
chr1:32387132,0.214333,0.507000,0.992000,0.968333,0.968667,0.965667,0.9420,0.981000,0.9585,0.8695,...,0.105103,0.101872,0.076385,0.118385,0.094154,-0.001692,0.052692,0.138846,0.054846,0.123769
chr1:32387376,0.127000,0.633000,0.946667,0.881333,1.000000,0.860000,0.8300,0.928000,0.7750,0.6930,...,0.223179,0.072410,0.040103,0.145641,-0.019128,-0.107436,-0.061128,0.113872,0.109564,0.110821
chr1:32387403,0.136667,0.869333,0.958333,0.948667,1.000000,0.953667,0.8635,1.000000,0.9690,0.7770,...,0.146551,0.096654,-0.000449,0.146551,0.113167,-0.093603,-0.164141,0.077090,0.132551,0.130038
chr1:32387409,0.182333,0.969667,0.989667,0.990667,0.982333,0.957333,0.9900,1.000000,1.0000,0.7570,...,0.097346,0.070423,0.105603,0.116372,0.116372,-0.145321,-0.155013,0.093218,0.103449,0.066474
chr1:32387417,0.666667,0.957333,0.980333,0.956333,1.000000,0.912667,1.0000,0.981333,0.9665,0.9300,...,0.063795,-0.030256,0.063795,0.043692,0.027718,-0.011590,-0.032590,0.023949,0.040103,0.029333


In [2]:

allcolumns=smdf.columns.tolist()
allcollength=len(allcolumns)


allct=[item for item in allcolumns if '-others' not in item]
allctlen=len(allct)

if allcollength!=2*allctlen:
    print('wrong all ct inferred. EXITING')
    sys.exit(1)

In [3]:
finaldf['#CpG']=-1

finaldf['deltabasedfragassignment']='NotAssigned'

score_columns=[]
for ct in allct:
    tempscorecol=ct+'-others'
    finaldf[tempscorecol]='NotAssigned'
    score_columns.append(tempscorecol)
finaldf.head()

,ReadName,allcelltype,ucelltype,assignedcelltype,CheckedCpG,mismatchCpG,notacceptedCpG,acceptedCpG,finalacceptedfor,finalrejectedfor,...,Mn-others,mNeu-others,m8-others,DC-others,Eo-others,Tr-others,m4-others,Mg-others,Er-others,mB-others
0,mNeu.HWI-ST699:245:C5RREACXX:1:1306:17784:57178,[],[],NotAssigned,[],[],[],[],notdetermined,notdetermined,...,NotAssigned,NotAssigned,NotAssigned,NotAssigned,NotAssigned,NotAssigned,NotAssigned,NotAssigned,NotAssigned,NotAssigned
1,m4.HWI-ST537:240:C6B3NACXX:3:2310:15483:54518,[],[],NotAssigned,[],[],[],[],notdetermined,notdetermined,...,NotAssigned,NotAssigned,NotAssigned,NotAssigned,NotAssigned,NotAssigned,NotAssigned,NotAssigned,NotAssigned,NotAssigned
2,mNeu.HWI-ST731_20:3:2106:7915:23475#0,[],[],NotAssigned,[],[],[],[],notdetermined,notdetermined,...,NotAssigned,NotAssigned,NotAssigned,NotAssigned,NotAssigned,NotAssigned,NotAssigned,NotAssigned,NotAssigned,NotAssigned
3,Tr.D6L3XBQ1:317:C6CGAACXX:6:2114:7476:15608,[NK],[NK],NotAssigned,[chr1:1221863],[],[chr1:1221863],[],notdetermined,notdetermined,...,NotAssigned,NotAssigned,NotAssigned,NotAssigned,NotAssigned,NotAssigned,NotAssigned,NotAssigned,NotAssigned,NotAssigned
4,Tr.HWI-ST700660:293:C737DACXX:6:2209:17216:67776,[NK],[NK],NotAssigned,[chr1:1221863],[],[chr1:1221863],[],notdetermined,notdetermined,...,NotAssigned,NotAssigned,NotAssigned,NotAssigned,NotAssigned,NotAssigned,NotAssigned,NotAssigned,NotAssigned,NotAssigned


In [4]:







def deltacalculation(cpgsfordelta):
    
    
    
    deltafortheseCpGS=cpgsfordelta[score_columns].sum(axis=0)

    return deltafortheseCpGS



def processrow(aindex,arow):
   

    notacceptedCpGList=arow[noACCcol]
        

    acceptedCpGList=arow[acccol]
    

    
    
    
    
    accindex=smdf.index.intersection(acceptedCpGList) #acceptedCpGList ##### 
    
    

    
    
    
    
    ACCscore=0
    if len(accindex)>0:
        corresACCcpgs=(smdf.loc[accindex])*-1 
        ACCscore=deltacalculation(corresACCcpgs)
    

  
    
    rejindex=smdf.index.intersection(notacceptedCpGList)  #notacceptedCpGList ##########
    

    
    
    
    REJscore=0
    if len(rejindex)>0:
        corresREJcpgs=smdf.loc[rejindex]
        
        REJscore=deltacalculation(corresREJcpgs)
    

    
    finalscore=ACCscore+REJscore
    


    
    finaldf.loc[index,score_columns]=finalscore #.iloc[0]
 
    
    totalcpgs=len(accindex)+len(rejindex)
    
    finaldf.loc[index,'#CpG']=totalcpgs

In [5]:
for index, row in finaldf.iterrows():
    

    processrow(index, row)

In [6]:
finaldf.head()

,ReadName,allcelltype,ucelltype,assignedcelltype,CheckedCpG,mismatchCpG,notacceptedCpG,acceptedCpG,finalacceptedfor,finalrejectedfor,...,Mn-others,mNeu-others,m8-others,DC-others,Eo-others,Tr-others,m4-others,Mg-others,Er-others,mB-others
0,mNeu.HWI-ST699:245:C5RREACXX:1:1306:17784:57178,[],[],NotAssigned,[],[],[],[],notdetermined,notdetermined,...,0,0,0,0,0,0,0,0,0,0
1,m4.HWI-ST537:240:C6B3NACXX:3:2310:15483:54518,[],[],NotAssigned,[],[],[],[],notdetermined,notdetermined,...,0,0,0,0,0,0,0,0,0,0
2,mNeu.HWI-ST731_20:3:2106:7915:23475#0,[],[],NotAssigned,[],[],[],[],notdetermined,notdetermined,...,0,0,0,0,0,0,0,0,0,0
3,Tr.D6L3XBQ1:317:C6CGAACXX:6:2114:7476:15608,[NK],[NK],NotAssigned,[chr1:1221863],[],[chr1:1221863],[],notdetermined,notdetermined,...,0,0,0,0,0,0,0,0,0,0
4,Tr.HWI-ST700660:293:C737DACXX:6:2209:17216:67776,[NK],[NK],NotAssigned,[chr1:1221863],[],[chr1:1221863],[],notdetermined,notdetermined,...,0,0,0,0,0,0,0,0,0,0


In [7]:

finaldf=finaldf[finaldf['#CpG']>0]

finaldf.head()

,ReadName,allcelltype,ucelltype,assignedcelltype,CheckedCpG,mismatchCpG,notacceptedCpG,acceptedCpG,finalacceptedfor,finalrejectedfor,...,Mn-others,mNeu-others,m8-others,DC-others,Eo-others,Tr-others,m4-others,Mg-others,Er-others,mB-others
1242,m4.HWI-ST539:248:C76MRACXX:5:1304:2280:33690,[CD4],[CD4],NotAssigned,[chr1:32387132],[],[chr1:32387132],[],notdetermined,notdetermined,...,0.105103,0.101872,0.0763846,0.118385,0.0941538,-0.00169231,0.0526923,0.138846,0.0548462,0.123769
1243,Mg.HWI-ST700660:289:C6N9FACXX:3:1108:11949:89241,[CD4],[CD4],NotAssigned,[chr1:32387132],[],[chr1:32387132],[],notdetermined,notdetermined,...,0.105103,0.101872,0.0763846,0.118385,0.0941538,-0.00169231,0.0526923,0.138846,0.0548462,0.123769
1244,Er.HWI-ST539_177:4:2111:14055:74502#8,[CD4],[CD4],NotAssigned,[chr1:32387132],[],[chr1:32387132],[],notdetermined,notdetermined,...,0.105103,0.101872,0.0763846,0.118385,0.0941538,-0.00169231,0.0526923,0.138846,0.0548462,0.123769
1245,Er.HWI-ST858_152:2:1214:12610:36368#8,[CD4],[CD4],NotAssigned,[chr1:32387132],[],[chr1:32387132],[],notdetermined,notdetermined,...,0.105103,0.101872,0.0763846,0.118385,0.0941538,-0.00169231,0.0526923,0.138846,0.0548462,0.123769
1246,CD8.HWI-ST459_144:7:1109:21110:33238#0,[CD4],[CD4],NotAssigned,[chr1:32387132],[],[chr1:32387132],[],notdetermined,notdetermined,...,0.105103,0.101872,0.0763846,0.118385,0.0941538,-0.00169231,0.0526923,0.138846,0.0548462,0.123769


In [8]:

ctpropdict= defaultdict(list)


for score in score_columns:
    

    
    posscore=finaldf.loc[finaldf[score]>deltagreaterforpositive,score]
    
    negscore=finaldf.loc[finaldf[score]<deltasmallerfornegative,score]
    
    
    
    poscpg=(finaldf.loc[finaldf[score]>deltagreaterforpositive,'#CpG']).sum()
    negcpg=(finaldf.loc[finaldf[score]<deltasmallerfornegative,'#CpG']).sum()
    
    
    poslen=len(posscore)
    neglen=len(negscore)
    
    posscoreSUM=sum(posscore)
    negscoreSUM=sum(negscore)
    
    
    
    ct_posnegscore=posscoreSUM/(posscoreSUM+abs(negscoreSUM))
    
    
    if poscpg>0:
        ct_posscorebyposcpg=posscoreSUM/poscpg
    else:
        ct_posscorebyposcpg=0
    
    ct_posscorebyallcpg=posscoreSUM/(poscpg+negcpg)
    
    ct_fragscore=poslen/(poslen+neglen)
    
    
    ctname=score.replace('-others','')
    
    ctpropdict[ctname+"_posnegscore"].append(ct_posnegscore)
    ctpropdict[ctname+"_posscorebyposcpg"].append(ct_posscorebyposcpg)
    ctpropdict[ctname+"_posscorebyallcpg"].append(ct_posscorebyallcpg)
    ctpropdict[ctname+"_fragscore"].append(ct_fragscore)
    
    
    
    
    
    

In [9]:
ctpropdf=pd.DataFrame.from_dict(ctpropdict)
ctpropdf.to_csv(finalfile+"_softRD_result.txt",sep="\t",index=False)

In [10]:
finaldf.to_pickle(finalfile+"_softRD.pkl")

In [11]:
end_time = time.time()

time_elapsed = (end_time - start_time)

print(time_elapsed)

355.33080196380615
